# ME 5514 : Final Project
##  Isolators Design for an Ultra Precision Manufacturing Machine
Calculation of Natural Frequency

In [ ]:
import sympy as smp
from scipy.integrate import odeint, solve_ivp
import numpy as np
import matplotlib.pyplot as plt

### Numerical Simulation Setup

In [ ]:
m, I, ky, kthe, h = smp.symbols('m, I ,ky ,kthe ,h')

M = smp.Matrix([[m,0],[0,I]])
K = smp.Matrix([[ky, -ky*h],[-ky*h, kthe+(ky*h**2)]])

M_sqrt = smp.sqrt(M)
M_inv = M_sqrt.inv()
K_til = M_inv @ K @ M_inv

lam = K_til.eigenvals()
wn = smp.sqrt(list(lam)[0])

In [ ]:
# Parameters

m = 1182              # mass (kg)
I = 96                # moment of inertia (kg m^2)
ky = 800000           # Stiffness in y (N/m)
ktheta = 90000        # Stiffness in rotation (Nm/rad)
cy = ky/100           # Damping in y (Kg/s)
ctheta = ktheta/100   # Damping in rotation (Nm-s/rad)
hf = 0.435            # Height of external force in y (m)
Fy = 50                # External force (N)
h = 0                 # Height from center of gravity (m)
freq = 0.1              # Forcing frequency (Hz)
w = 2 * np.pi * freq  # Forcing frequency (rad)
Y = 0.005             # Amplitude of external excitation (m)
para = np.array([m, I, ky, ktheta, cy, ctheta, hf, w, Fy, h, Y])

# State Space function

def dudt(t,x,m, I, ky, ktheta, cy, ctheta, hf, w, Fy, h, Y):
        
    M = np.array([[m,0],
                  [0,I]])
    
    C = np.array([[cy, -cy*h],
                  [-cy*h, ctheta+cy*h**2]])
    
    K = np.array([[ky, -ky*h],
                  [-ky*h, ktheta+ky*h**2]])
    
    
    third_term = -np.linalg.inv(M) @ K
    fourth_term = -np.linalg.inv(M) @ C
    
    A1 = np.block([[np.zeros((2,2)),np.eye(2)],[third_term,fourth_term]])
    A2 = np.block([[np.zeros((2,4))],[np.linalg.inv(M) @ K,np.linalg.inv(M) @ C]])
    
    u0 = np.array([[Y*np.sin(w*t), 0]]).T
    u0_dot = np.array([[w*Y*np.cos(w*t), 0]]).T
    
    # External force
    B = np.array([[Fy,-Fy*hf]])
    f = np.linalg.inv(M) @ B.T
    f1 = np.concatenate([np.zeros(2),f.ravel()])
    
    u_comb =  np.concatenate([u0.ravel(),u0_dot.ravel()])
    
    v = A1@x + f1*np.sin(w*t) + A2 @ u_comb
    
    return v

#Initial conditions

u1_0 = 0
u2_0 = 0
u1_0_dot = 0
u2_0_dot = 0
U_0 = [u1_0, u2_0, u1_0_dot, u2_0_dot]

# Solving the system of equation
t = np.linspace(0,20,601)
sol = solve_ivp(dudt, t_span=(0,max(t)), y0 = U_0, t_eval=t, args=para)

### Calculation the response of the system for various translational stiffness and damping

In [ ]:
freq_range = np.arange(0.1,8.0,0.1)
max_disp = np.zeros(np.size(freq_range))
disp_tran = np.zeros(np.size(freq_range))
ky_range = np.linspace(700000, 1000000, 4)
ky_disp_tran = np.zeros((np.size(freq_range),np.size(ky_range)))

j=0
for ky in ky_range:
    cy = ky/100           # Damping in y (Kg/s)
    i = 0
    for freq in freq_range:
        w = 2 * np.pi * freq  # Forcing frequency (rad)
        para = np.array([m, I, ky, ktheta, cy, ctheta, hf, w, Fy, h, Y])
        sol = solve_ivp(dudt, t_span=(0,max(t)), y0 = U_0, t_eval=t, args=para, rtol=1e-11, atol=1e-9)
        y_disp = sol.y[0]
        max_disp[i] = max(y_disp[300::])
        disp_tran[i] = max_disp[i] / Y
        i += 1
    ky_disp_tran[:,j] = disp_tran
    j += 1   

### Calculation the response of the system for various rotational stiffness and damping

In [ ]:
freq_range = np.arange(0.1,8.0,0.05)
max_disp = np.zeros(np.size(freq_range))
disp_tran = np.zeros(np.size(freq_range))
kth_range = np.linspace(80000, 120000, 5)
kth_disp_tran = np.zeros((np.size(freq_range),np.size(kth_range)))

j=0
for ktheta in kth_range:
    ctheta = ktheta/100           
    i = 0
    for freq in freq_range:
        w = 2 * np.pi * freq  # Forcing frequency (rad)
        para = np.array([m, I, ky, ktheta, cy, ctheta, hf, w, Fy, h, Y])
        sol = solve_ivp(dudt, t_span=(0,max(t)), y0 = U_0, t_eval=t, args=para, rtol=1e-11, atol=1e-9)
        y_disp = sol.y[1]
        max_disp[i] = max(y_disp[100::])
        #disp_tran[i] = max_disp[i] / Y
        i += 1
    kth_disp_tran[:,j] = max_disp
    j += 1

### Calculation the response of the system for varying location of isolator

In [ ]:
h_range = np.linspace(-0.2,0.2,9)
h_disp_tran = np.zeros((np.size(freq_range),np.size(h_range)))

j=0
for h in h_range:           
    i = 0
    for freq in freq_range:
        w = 2 * np.pi * freq  # Forcing frequency (rad)
        para = np.array([m, I, ky, ktheta, cy, ctheta, hf, w, Fy, h, Y])
        sol = solve_ivp(dudt, t_span=(0,max(t)), y0 = U_0, t_eval=t, args=para, rtol=1e-11, atol=1e-9)
        y_disp = sol.y[0]
        max_disp[i] = max(y_disp[300::])
        disp_tran[i] = max_disp[i] / Y
        i += 1
    h_disp_tran[:,j] = disp_tran
    j += 1

### Comparison of Analytical and Numerical solution

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_figwidth(10)
fig.set_figheight(8)
plt.plot(freq_range,ana_force_trans,label='Analytical ($F_y=100~N~&~h=0~mm$)')
plt.plot(freq_range,disp_tran1,label='Numerical ($F_y=100~N~&~h=0~mm$)')
plt.plot(freq_range,ana_tran1,label='Analytical ($F_y=0~N~&~h=0~mm$)')
plt.plot(freq_range,disp_tran2,label='Numerical ($F_y=0~N~&~h=0~mm$)')
plt.title('Verification of Analytical and Numerical solution')
plt.xlabel('Frequency, $Hz$')
plt.ylabel('Transmissibility')
plt.legend(loc='upper right')
plt.show()
fig.savefig("ana_num_f", bbox_inches="tight", transparent=True)

### Response of the system for various translational stiffness

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_figwidth(10)
fig.set_figheight(8)
plt.plot(freq_range1, ky_run1[:,0],label='$k_y$ = 700,000 N/m')
plt.plot(freq_range1, ky_run1[:,1],label='$k_y$ = 800,000 N/m')
plt.plot(freq_range1, ky_run1[:,2],label='$k_y$ = 900,000 N/m')
plt.plot(freq_range1, ky_run1[:,3],label='$k_y$ = 1,000,000 N/m')
plt.title('Transmissibility for various Translational Stiffness ($k_y$)')
plt.xlabel('Frequency, $Hz$')
plt.ylabel('Transmissibility')
plt.legend(loc='upper right')
plt.show()
fig.savefig("translational_stiffenss", bbox_inches="tight", transparent=True)

### Response of the system for various translational damping

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_figwidth(10)
fig.set_figheight(8)
plt.plot(freq_range, ky_disp_tran[:,0],label='${\zeta}_y$ = 0.122')
plt.plot(freq_range, ky_disp_tran[:,1],label='${\zeta}_y$ = 0.130')
plt.plot(freq_range, ky_disp_tran[:,2],label='${\zeta}_y$ = 0.138')
plt.plot(freq_range, ky_disp_tran[:,3],label='${\zeta}_y$ = 0.145')
plt.title('Transmissibility for various Translational Damping Ratio (${\zeta}_y$)')
plt.xlabel('Frequency, $Hz$')
plt.ylabel('Transmissibility')
plt.legend(loc='upper right')
plt.show()
fig.savefig("translational_damping", bbox_inches="tight", transparent=True)

### Response of the system for various rotational stiffness

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_figwidth(10)
fig.set_figheight(8)
plt.plot(freq_range, kth_disp_tran[:,0],label='$k_{\Theta}$ = 80,000 Nm/rad')
plt.plot(freq_range, kth_disp_tran[:,1],label='$k_{\Theta}$ = 90,000 Nm/rad')
plt.plot(freq_range, kth_disp_tran[:,2],label='$k_{\Theta}$ = 100,000 Nm/rad')
plt.plot(freq_range, kth_disp_tran[:,3],label='$k_{\Theta}$ = 110,000 Nm/rad')
plt.plot(freq_range, kth_disp_tran[:,4],label='$k_{\Theta}$ = 120,000 Nm/rad')
plt.title('Frequency response for various Rotational Stiffness ($k_{\Theta}$)')
plt.xlabel('Frequency, $Hz$')
plt.ylabel('Response,$rad$')
plt.legend(loc='upper right')
plt.show()
fig.savefig("rotational_stiffness", bbox_inches="tight", transparent=True)

### Response of the system for various rotational damping

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_figwidth(10)
fig.set_figheight(8)
plt.plot(freq_range, kth_disp_tran[:,0],label='${\zeta}_{\Theta}$ = 0.144')
plt.plot(freq_range, kth_disp_tran[:,1],label='${\zeta}_{\Theta}$ = 0.153')
plt.plot(freq_range, kth_disp_tran[:,2],label='${\zeta}_{\Theta}$ = 0.161')
plt.plot(freq_range, kth_disp_tran[:,3],label='${\zeta}_{\Theta}$ = 0.169')
plt.plot(freq_range, kth_disp_tran[:,4],label='${\zeta}_{\Theta}$ = 0.177')
plt.title('Frequency response for various Rotational Damping Ratio (${\zeta}_{\Theta}$)')
plt.xlabel('Frequency, $Hz$')
plt.ylabel('Response,$rad$')
plt.legend(loc='upper right')
plt.show()
fig.savefig("rotational_damping", bbox_inches="tight", transparent=True)

### Response of the system for various location of isolator

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
fig.set_figwidth(10)
fig.set_figheight(8)
plt.plot(freq_range, h_disp_tran[:,0],label='$h$ = -200 mm')
plt.plot(freq_range, h_disp_tran[:,1],label='$h$ = -100 mm')
plt.plot(freq_range, h_disp_tran[:,2],label='$h$ = -0 mm')
plt.plot(freq_range, h_disp_tran[:,3],label='$h$ = 100 mm')
plt.plot(freq_range, h_disp_tran[:,4],label='$h$ = 200 mm')
plt.title('Transmissibility for various isolator location from C.G ($h$)')
plt.xlabel('Frequency, $Hz$')
plt.ylabel('Transmissibility')
plt.legend(loc='upper right')
plt.show()
fig.savefig("iso_height", bbox_inches="tight", transparent=True)